clean:
abc: 0.9109
cnn: 0.9097

pgd20
alpha=0.1
abc: 0.3411
cnn: 0.3450

alpha=1
abc: 0.6949
cnn: 0.6882

alpha=0.15
abc: 0.7582
cnn: 0.7490

pgd50
abc: 0.7283
cnn: 0.7211

fgsm
abc: 0.3871
cnn: 0.3540


foolbox
fgsm: epsilon=0.031
abc: 0.1400
cnn: 0.2149

fgsm: epsilon=0.005
abccnn: 0.6807
abc: 0.4621
cnn: 0.4154

pgd20: alpha=0.1 epsilon=0.005
abccnn: 0.4328
abc: 0.2401
cnn: 0.0970


ABCCNN
clean    0.9363
fgsm     0.6807
pgd      0.4328

In [2]:
import torch
import torchvision
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils.tools import dotdict
from driver.driver import ABC_Driver
# from torch_geometric_temporal import METRLADatasetLoader
from other_model.other_model import make_default_model
import atd2022
torch.cuda.is_available()
torch.cuda.set_device(0)

In [2]:
cifar10_args = dotdict()

cifar10_args.name = 'cifar10'
cifar10_args.train_batch_size = 128
cifar10_args.predict_batch_size = 128
cifar10_args.device = ['cuda:0']

cifar10_args.train_epochs = 250
cifar10_args.lr = 0.01
cifar10_args.criterion = 'CE'
cifar10_args.optimizer = 'AdamW'
cifar10_args.scheduler = 'OneCycle'
cifar10_args.attack = {'fgsm':(0.005,), 'pgd':(0.005,0.1,20),'deepfool':(0.005,0.1,20),'apgd-ce':(0.005,)}

activation = 'relu'
input_channel = 3
knpp = [40,80,120,160,240,320,400,480,560]
groups=40

cifar10_args.layers=[
    ('cnn2d', ((input_channel, knpp[0], (3,3), 1, 1, 1, 1), 1, None, None, activation, False)),
    ('atrc2d', ((knpp[0], knpp[1], (3,3), 1, 1, 1, groups), 1, None, None, activation, False)),
    ('atrc2d', ((knpp[1], knpp[2], (3,3), 1, 1, 1, groups), 1, None, None, activation, False)),
    ('atrc2d', ((knpp[2], knpp[3], (3,3), 1, 1, 1, groups), 1, 'first', (2,2), activation, False)),
    ('atrc2d', ((knpp[3], knpp[4], (3,3), 1, 1, 1, groups), 1, None, None, activation, False)),
    ('atrc2d', ((knpp[4], knpp[5], (3,3), 1, 1, 1, groups), 1, None, None, activation, False)),
    ('atrc2d', ((knpp[5], knpp[6], (3,3), 1, 1, 1, groups), 1, 'first', (2,2), activation, False)),
    ('atrc2d', ((knpp[6], knpp[7], (3,3), 1, 1, 1, groups), 1, None, None, activation, False)),
    ('atrc2d', ((knpp[7], knpp[8], (3,3), 1, 1, 1, groups), 1, None, None, activation, False)),
    ('adptavgpool', (1,1)), 
    ('linear', (knpp[-1], 100, (1,2,3)))
]

In [3]:
driver = ABC_Driver(cifar10_args, None, record_path=None, if_hash=False)
driver.train()

Use: ['cuda:0']
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
add record: 03/27/2023 12:02
epoch: 0, train_loss: 1.8453, test_metric: 0.5314, time: 731.6992020606995
epoch: 1, train_loss: 1.4178, test_metric: 0.6197, time: 730.3269581794739
epoch: 2, train_loss: 1.2223, test_metric: 0.65, time: 730.6451373100281
epoch: 3, train_loss: 1.1058, test_metric: 0.7094, time: 730.5678844451904
epoch: 4, train_loss: 1.009, test_metric: 0.6845, time: 732.0547330379486
epoch: 5, train_loss: 0.9501, test_metric: 0.7464, time: 732.7172274589539
epoch: 6, train_loss: 0.886, test_metric: 0.7285, time: 732.5388605594635
epoch: 7, train_loss: 0.848, test_metric: 0.6882, time: 732.6343474388123
epoch: 8, train_loss: 0.8114, test_metric: 0.7425, time: 732.922384262085
epoch: 9, train_loss: 0.7717, test_metric: 0.7378, time: 733.4980013370514
epoch: 10, train_loss: 0.7574, test_metric: 0.7839, time: 733.1797575950623
epoch: 11, train_loss

epoch: 108, train_loss: 0.1079, test_metric: 0.942, time: 697.5037336349487
epoch: 109, train_loss: 0.1029, test_metric: 0.9443, time: 697.4538731575012
epoch: 110, train_loss: 0.1056, test_metric: 0.9407, time: 697.4034421443939
epoch: 111, train_loss: 0.1044, test_metric: 0.9374, time: 697.4445436000824
epoch: 112, train_loss: 0.1016, test_metric: 0.9375, time: 697.657265663147
epoch: 113, train_loss: 0.1031, test_metric: 0.9413, time: 698.395589351654
epoch: 114, train_loss: 0.0992, test_metric: 0.9378, time: 700.355633020401
epoch: 115, train_loss: 0.1012, test_metric: 0.9426, time: 700.3838155269623
epoch: 116, train_loss: 0.0991, test_metric: 0.9407, time: 700.2000677585602
epoch: 117, train_loss: 0.0966, test_metric: 0.933, time: 700.1348671913147
epoch: 118, train_loss: 0.0943, test_metric: 0.9423, time: 700.1909947395325
epoch: 119, train_loss: 0.0943, test_metric: 0.9411, time: 700.1229360103607
epoch: 120, train_loss: 0.0884, test_metric: 0.9414, time: 700.1937491893768
epoc

epoch: 217, train_loss: 0.0329, test_metric: 0.9536, time: 700.005752325058
epoch: 218, train_loss: 0.0335, test_metric: 0.9539, time: 700.2168321609497
epoch: 219, train_loss: 0.0317, test_metric: 0.9539, time: 700.1229050159454
epoch: 220, train_loss: 0.0315, test_metric: 0.954, time: 700.0949003696442
epoch: 221, train_loss: 0.0314, test_metric: 0.9532, time: 700.1822876930237
epoch: 222, train_loss: 0.0324, test_metric: 0.9527, time: 700.1228122711182
epoch: 223, train_loss: 0.0328, test_metric: 0.9533, time: 700.0319769382477
epoch: 224, train_loss: 0.0317, test_metric: 0.9522, time: 700.2214684486389
epoch: 225, train_loss: 0.033, test_metric: 0.9539, time: 700.0872621536255
epoch: 226, train_loss: 0.0315, test_metric: 0.9542, time: 700.084091424942
epoch: 227, train_loss: 0.0327, test_metric: 0.9533, time: 700.2473497390747
epoch: 228, train_loss: 0.0316, test_metric: 0.9538, time: 699.9895298480988
epoch: 229, train_loss: 0.031, test_metric: 0.9532, time: 700.0467731952667
epoc

OutOfMemoryError: CUDA out of memory. Tried to allocate 3.56 GiB (GPU 0; 47.46 GiB total capacity; 30.77 GiB already allocated; 2.06 GiB free; 44.34 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [3]:
driver = ABC_Driver(cifar10_args, None, record_path=None, if_hash=False)
driver.train()

Use: ['cuda:0']
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
add record: 03/25/2023 16:31
epoch: 0, train_loss: 1.7716, test_metric: 0.4988, time: 593.2482035160065
epoch: 1, train_loss: 1.4845, test_metric: 0.5628, time: 591.7711906433105
epoch: 2, train_loss: 1.3432, test_metric: 0.6113, time: 591.7671165466309
epoch: 3, train_loss: 1.2384, test_metric: 0.6279, time: 591.7860770225525
epoch: 4, train_loss: 1.1637, test_metric: 0.6348, time: 591.573098897934
epoch: 5, train_loss: 1.0978, test_metric: 0.6952, time: 591.6569447517395
epoch: 6, train_loss: 1.0395, test_metric: 0.7138, time: 592.107753276825
epoch: 7, train_loss: 1.0017, test_metric: 0.7064, time: 592.1148927211761
epoch: 8, train_loss: 0.9595, test_metric: 0.7341, time: 591.9846529960632
epoch: 9, train_loss: 0.9299, test_metric: 0.7449, time: 591.9319047927856
epoch: 10, train_loss: 0.8987, test_metric: 0.745, time: 591.9915819168091
epoch: 11, train_l

epoch: 107, train_loss: 0.1207, test_metric: 0.9359, time: 592.8108556270599
epoch: 108, train_loss: 0.1132, test_metric: 0.936, time: 592.8189835548401
epoch: 109, train_loss: 0.1181, test_metric: 0.937, time: 592.642333984375
epoch: 110, train_loss: 0.1155, test_metric: 0.9385, time: 593.0039255619049
epoch: 111, train_loss: 0.1119, test_metric: 0.9386, time: 593.1877944469452
epoch: 112, train_loss: 0.1105, test_metric: 0.9365, time: 593.3193695545197
epoch: 113, train_loss: 0.1085, test_metric: 0.9378, time: 592.9271106719971
epoch: 114, train_loss: 0.1099, test_metric: 0.9408, time: 592.958417892456
epoch: 115, train_loss: 0.1069, test_metric: 0.9383, time: 592.8823490142822
epoch: 116, train_loss: 0.1064, test_metric: 0.9447, time: 592.7913644313812
epoch: 117, train_loss: 0.1075, test_metric: 0.9388, time: 592.788667678833
epoch: 118, train_loss: 0.1008, test_metric: 0.9354, time: 592.8589706420898
epoch: 119, train_loss: 0.1001, test_metric: 0.9374, time: 592.8055758476257
epoc

epoch: 215, train_loss: 0.036, test_metric: 0.9517, time: 592.6178703308105
epoch: 216, train_loss: 0.0367, test_metric: 0.9502, time: 592.8845608234406
epoch: 217, train_loss: 0.0345, test_metric: 0.9503, time: 592.8010311126709
epoch: 218, train_loss: 0.0362, test_metric: 0.9524, time: 592.9475555419922
epoch: 219, train_loss: 0.0349, test_metric: 0.9512, time: 592.7571353912354
epoch: 220, train_loss: 0.0356, test_metric: 0.9507, time: 592.9515450000763
epoch: 221, train_loss: 0.0374, test_metric: 0.9514, time: 592.6132092475891
epoch: 222, train_loss: 0.0334, test_metric: 0.9501, time: 592.8214802742004
epoch: 223, train_loss: 0.0336, test_metric: 0.9512, time: 592.7015759944916
epoch: 224, train_loss: 0.0353, test_metric: 0.952, time: 591.5550420284271
epoch: 225, train_loss: 0.0337, test_metric: 0.9529, time: 593.0653738975525
epoch: 226, train_loss: 0.0335, test_metric: 0.953, time: 593.0131728649139
epoch: 227, train_loss: 0.035, test_metric: 0.9519, time: 592.7312452793121
epo

In [6]:
torch.save(driver.model.state_dict(), "save/CIFAR10_ARC_2023_03_27_09522.pt")
# driver.model.load_state_dict(torch.load("save/CIFAR10_ABC_2023_03_07.pt"))